In [1]:
from os import listdir
import numpy as np

from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

In [10]:
import tensorflow as tf

In [11]:
len(tf.config.experimental.list_physical_devices('GPU'))

0

In [2]:
path_to_input = './dataset/areas/'
path_to_target = './dataset/buildings/'
    
def load_images():
    return load_images_from_path(path_to_input), load_images_from_path(path_to_target)
    

def load_images_from_path(path):
    data = list()
    for filename in listdir(path):
        # load and resize the image
        pixels = load_img(path + filename)
        # convert to numpy array
        pixels = img_to_array(pixels)
        data.append(pixels)
    return np.asarray(data)

In [3]:
path = './dataset/areas/'
img_to_array(load_img(path + 'area_0.png')).shape

(192, 256, 3)

In [4]:
data_input, data_target = load_images()
np.savez_compressed('./dataset/dataset.npz', data_input, data_target)
data_input.shape, data_target.shape

KeyboardInterrupt: 

((5089, 160, 240, 3), (5089, 160, 240, 3))